In [1]:
import os
import cv2
import random
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, Lambda, Dense, Flatten, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import tensorflow as tf
from sklearn.model_selection import train_test_split


In [2]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
def preprocess_image(image_path, target_size=(128,128)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,target_size)
    img = img/255.0
    return img

## Creating image triplets 

In [4]:
def create_image_triplets(whale_folder_path, num_pairs_per_id = 10, img_size=(128,128)):
    whale_ids = os.listdir(whale_folder_path)
#     anchor = []
#     postive_pairs = []
#     negative_pairs = []
    triplets = []
    
    #m7tgeen n3ml el anchor wl pos wl neg mn different folder 
    
    for whale_id in whale_ids:
        whale_images_path = os.path.join(whale_folder_path, whale_id)
        whale_images = [os.path.join(whale_images_path, img) for img in os.listdir(whale_images_path) if img.endswith('.jpg')]
        number_of_whales = len(whale_images)
        
        for i in range(num_pairs_per_id):
            anchor, positive = random.sample(whale_images,2)
            negative_whale_id = random.choice([id for id in whale_ids if id != whale_id])
            negative_whale_images_path = os.path.join(whale_folder_path,negative_whale_id)
            negative = random.choice([os.path.join(negative_whale_images_path, img) for img in os.listdir(negative_whale_images_path) if img.endswith('.jpg')])
            
            #ekhtarna random sets n append ll triples
            
            triplets.append((anchor,positive,negative))
            random.shuffle(triplets)
    print(len(triplets))
    return triplets

## Preprocessing the triplets images

In [5]:
def preprocess_triplets(triplets, img_size=(128, 128)):
    anchors, positives, negatives = [], [], []

    for anchor_path, positive_path, negative_path in triplets:
        anchor = preprocess_image(anchor_path, img_size)
        positive = preprocess_image(positive_path, img_size)
        negative = preprocess_image(negative_path, img_size)

        anchors.append(anchor)
        positives.append(positive)
        negatives.append(negative)

    return np.array(anchors), np.array(positives), np.array(negatives)




## Create Pairs with Labels

In [6]:
def create_pairs(anchors, positives, negatives):
    pairs = []
    labels = []

    for i in range(len(anchors)):
        pairs += [[anchors[i], positives[i]]]
        labels += [1.0]  # Cast label as float

    for i in range(len(anchors)):
        pairs += [[anchors[i], negatives[i]]]
        labels += [0.0]  # Cast label as float

    return np.array(pairs), np.array(labels)

In [18]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

def create_base_network(input_shape):
    input = Input(shape=input_shape)
    x = Conv2D(64, (10,10), activation='relu')(input)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Conv2D(128, (7,7), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Conv2D(128, (4,4), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Conv2D(256, (4,4), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Flatten()(x)
    x = Dense(4096, activation='sigmoid')(x)
    return Model(input, x)

input_shape = (128, 128, 3)
base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))([processed_a, processed_b])
output = Dense(1, activation='sigmoid')(distance)

model = Model([input_a, input_b], output)

model.compile(loss='binary_crossentropy', optimizer=Adam(0.0001), metrics=['accuracy'])

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_10 (InputLayer)          [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 model_2 (Functional)           (None, 4096)         17988928    ['input_9[0][0]',                
                                                                  'input_10[0][0]']         

In [ ]:
test = r"C:\Users\islam\Downloads\Dataset\Dataset\few-shot-learning\train\whale"
triplets = create_image_triplets(test) 
anchors, positives, negatives = preprocess_triplets(triplets)
pairs, labels = create_pairs(anchors, positives, negatives)

# Ensure pairs is a list of two numpy arrays
pairs = [np.array([pair[0] for pair in pairs]), np.array([pair[1] for pair in pairs])]

# Ensure labels are of the correct shape
labels = np.array(labels).reshape(-1, 1)

# Train the model
model.fit(pairs, labels, batch_size=5, epochs=15)


500
Epoch 1/15
 11/200 [>.............................] - ETA: 6s - loss: 0.6929 - accuracy: 0.5091

In [9]:
def select_reference_images(train_folder):
    reference_images = {}
    whale_ids = os.listdir(train_folder)
    print("whale ids are",whale_ids)

    for whale_id in whale_ids:
        whale_folder = os.path.join(train_folder, whale_id)
        for img_file in os.listdir(whale_folder):
            if img_file.endswith('.jpg'):
                img_path = os.path.join(whale_folder, img_file)
                reference_images[whale_id] = preprocess_image(img_path)
                break  # Select the first image as reference and move to next ID

    return reference_images


In [10]:
def load_validation_images(validation_folder):
    validation_images = []
    image_paths = []

    for img_file in os.listdir(validation_folder):
        if img_file.endswith('.jpg'):
            img_path = os.path.join(validation_folder, img_file)
            img = preprocess_image(img_path)
            validation_images.append((img, img_path))

    return validation_images


In [13]:
def predict_whale_id(model, test_image, reference_images):
    max_similarity = 0
    predicted_id = None

    for whale_id, ref_image in reference_images.items():
        similarity = model.predict([np.expand_dims(test_image, axis=0), np.expand_dims(ref_image, axis=0)])
        print("similarity is " , similarity, "whale id is",whale_id)
        if similarity > max_similarity:
            max_similarity = similarity
            predicted_id = whale_id
    return predicted_id

# Assuming model is your trained Siamese model
train_folder = r"C:\Users\islam\Downloads\Dataset\Dataset\few-shot-learning\val\whale"
validation_folder = r"C:\Users\islam\Downloads\New folder (2)"

reference_images = select_reference_images(train_folder)
validation_images = load_validation_images(validation_folder)

for test_image, test_image_path in validation_images:
    predicted_id = predict_whale_id(model, test_image, reference_images)
    print(f"Image {test_image_path} is predicted to be Whale ID: {predicted_id}")


whale ids are ['51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67']
1/1 [==============================] - 0s 22ms/step
similarity is  [[0.09120983]] whale id is 51
1/1 [==============================] - 0s 22ms/step
similarity is  [[0.00363087]] whale id is 52
1/1 [==============================] - 0s 22ms/step
similarity is  [[0.00025228]] whale id is 53
1/1 [==============================] - 0s 24ms/step
similarity is  [[0.4326236]] whale id is 54
1/1 [==============================] - 0s 23ms/step
similarity is  [[0.55875623]] whale id is 55
1/1 [==============================] - 0s 21ms/step
similarity is  [[0.27983573]] whale id is 56
1/1 [==============================] - 0s 23ms/step
similarity is  [[0.08193381]] whale id is 57
1/1 [==============================] - 0s 27ms/step
similarity is  [[0.44255915]] whale id is 58
1/1 [==============================] - 0s 24ms/step
similarity is  [[0.10400219]] whale id is 59
1/1 [======